<a href="https://colab.research.google.com/github/PeymanKazemi/NLP_course2021_2/blob/main/Fine_tune_Sick_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [77]:
!pip install datasets

In [2]:
!pip install -U transformers

     |████████████████████████████████| 3.4 MB 5.1 MB/s 
     |████████████████████████████████| 3.3 MB 52.5 MB/s 
     |████████████████████████████████| 895 kB 72.2 MB/s 
     |████████████████████████████████| 596 kB 74.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [84]:
from datasets import load_dataset
raw_dataset = load_dataset('sick')

Using custom data configuration default
Reusing dataset sick (/root/.cache/huggingface/datasets/sick/default/0.0.0/c6b3b0b44eb84b134851396d6d464e5cb8f026960519d640e087fe33472626db)


  0%|          | 0/3 [00:00<?, ?it/s]

In [132]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'sentence_A', 'sentence_B', 'label', 'relatedness_score', 'entailment_AB', 'entailment_BA', 'sentence_A_original', 'sentence_B_original', 'sentence_A_dataset', 'sentence_B_dataset'],
        num_rows: 4439
    })
    validation: Dataset({
        features: ['id', 'sentence_A', 'sentence_B', 'label', 'relatedness_score', 'entailment_AB', 'entailment_BA', 'sentence_A_original', 'sentence_B_original', 'sentence_A_dataset', 'sentence_B_dataset'],
        num_rows: 495
    })
    test: Dataset({
        features: ['id', 'sentence_A', 'sentence_B', 'label', 'relatedness_score', 'entailment_AB', 'entailment_BA', 'sentence_A_original', 'sentence_B_original', 'sentence_A_dataset', 'sentence_B_dataset'],
        num_rows: 4906
    })
})

In [133]:
new_dataset = raw_dataset.remove_columns([ 'sentence_A_original', 'sentence_B_original', 'sentence_A_dataset', 'sentence_B_dataset'])
new_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'sentence_A', 'sentence_B', 'label', 'relatedness_score', 'entailment_AB', 'entailment_BA'],
        num_rows: 4439
    })
    validation: Dataset({
        features: ['id', 'sentence_A', 'sentence_B', 'label', 'relatedness_score', 'entailment_AB', 'entailment_BA'],
        num_rows: 495
    })
    test: Dataset({
        features: ['id', 'sentence_A', 'sentence_B', 'label', 'relatedness_score', 'entailment_AB', 'entailment_BA'],
        num_rows: 4906
    })
})

In [143]:
def sum_of_AB(example):
  example['entailment_BA'] =  example['sentence_A'] + "'" + ', '+   "'" + example['sentence_B']
  return example

updated_dataset = new_dataset.map(sum_of_AB)
updated_dataset['train']['entailment_BA'][0]

  0%|          | 0/4439 [00:00<?, ?ex/s]

  0%|          | 0/495 [00:00<?, ?ex/s]

  0%|          | 0/4906 [00:00<?, ?ex/s]

"A group of kids is playing in a yard and an old man is standing in the background', 'A group of boys in a yard is playing and a man is standing in the background"

In [94]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("eugenesiow/bart-paraphrase")

In [110]:
def tokenizer_function(example):
  return tokenizer(example['sentence_A'], padding="max_length", truncation=True)


In [4]:
import pandas as pd

df_train = pd.DataFrame(data = raw_dataset['train'])
df_test = pd.DataFrame(data = raw_dataset['test'])
df_validation = pd.DataFrame(data = raw_dataset['validation'])

In [5]:
# merge dataframes
df_general = pd.concat([df_train, df_test, df_validation])
df_general.head()

,id,sentence_A,sentence_B,label,relatedness_score,entailment_AB,entailment_BA,sentence_A_original,sentence_B_original,sentence_A_dataset,sentence_B_dataset
0,1,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,1,4.5,A_neutral_B,B_neutral_A,"A group of children playing in a yard, a man i...","A group of children playing in a yard, a man i...",FLICKR,FLICKR
1,2,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,1,3.2,A_contradicts_B,B_neutral_A,"A group of children playing in a yard, a man i...","A group of children playing in a yard, a man i...",FLICKR,FLICKR
2,3,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,0,4.7,A_entails_B,B_entails_A,"The children are playing outdoors, while a man...","The children are playing outdoors, while a man...",FLICKR,FLICKR
3,5,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,1,3.4,A_neutral_B,B_neutral_A,"A group of children playing in a yard, a man i...","The children are playing outdoors, while a man...",FLICKR,FLICKR
4,9,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,1,3.7,A_neutral_B,B_neutral_A,"A group of children playing in a yard, a man i...","The children are playing outdoors, while a man...",FLICKR,FLICKR


In [6]:
refined_df = df_general[['sentence_A', 'sentence_B','label', 'relatedness_score', 'entailment_AB']]
refined_df.head()

,sentence_A,sentence_B,label,relatedness_score,entailment_AB
0,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,1,4.5,A_neutral_B
1,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,1,3.2,A_contradicts_B
2,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,0,4.7,A_entails_B
3,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,1,3.4,A_neutral_B
4,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,1,3.7,A_neutral_B


In [7]:
import math
def sigmoid(x): 
    return 1 / (1 + math.exp(-x))


refined_df['label_new'] = refined_df.relatedness_score.apply(sigmoid)
refined_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,sentence_A,sentence_B,label,relatedness_score,entailment_AB,label_new
0,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,1,4.5,A_neutral_B,0.989013
1,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,1,3.2,A_contradicts_B,0.960834
2,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,0,4.7,A_entails_B,0.990987
3,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,1,3.4,A_neutral_B,0.967705
4,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,1,3.7,A_neutral_B,0.975873
...,...,...,...,...,...,...
490,A biker is riding away from a fence,A man is dancing on the road,1,1.2,A_neutral_B,0.768525
491,A woman is playing an electric guitar,A kid is playing guitar,1,3.0,A_neutral_B,0.952574
492,The animal is grazing on the grass,The cop is sitting on a police bike,1,1.0,A_neutral_B,0.731059
493,The dog is snapping at some droplets of water,A girl in a band is playing an instrument,1,1.0,A_neutral_B,0.731059


In [8]:
refined_df.reset_index(drop=True, inplace=True)
refined_df

,sentence_A,sentence_B,label,relatedness_score,entailment_AB,label_new
0,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,1,4.5,A_neutral_B,0.989013
1,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,1,3.2,A_contradicts_B,0.960834
2,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,0,4.7,A_entails_B,0.990987
3,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,1,3.4,A_neutral_B,0.967705
4,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,1,3.7,A_neutral_B,0.975873
...,...,...,...,...,...,...
9835,A biker is riding away from a fence,A man is dancing on the road,1,1.2,A_neutral_B,0.768525
9836,A woman is playing an electric guitar,A kid is playing guitar,1,3.0,A_neutral_B,0.952574
9837,The animal is grazing on the grass,The cop is sitting on a police bike,1,1.0,A_neutral_B,0.731059
9838,The dog is snapping at some droplets of water,A girl in a band is playing an instrument,1,1.0,A_neutral_B,0.731059


In [9]:
dataset = refined_df[['sentence_A', 'sentence_B', 'label_new']]
dataset

,sentence_A,sentence_B,label_new
0,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,0.989013
1,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,0.960834
2,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,0.990987
3,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,0.967705
4,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,0.975873
...,...,...,...
9835,A biker is riding away from a fence,A man is dancing on the road,0.768525
9836,A woman is playing an electric guitar,A kid is playing guitar,0.952574
9837,The animal is grazing on the grass,The cop is sitting on a police bike,0.731059
9838,The dog is snapping at some droplets of water,A girl in a band is playing an instrument,0.731059


In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("eugenesiow/bart-paraphrase")

# tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/332 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [11]:
item_sentence  = [["Hello, I'm a single sentence! Thist great.","I linke it. At the most."],
                          ["test bacht2. test", "test2. test"]]


In [12]:
inputs = tokenizer(item_sentence, padding="max_length", truncation=True)
inputs

{'input_ids': [[0, 31414, 6, 38, 437, 10, 881, 3645, 328, 2032, 661, 372, 4, 2, 2, 100, 24248, 1071, 24, 4, 497, 5, 144, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
# return one column

In [13]:
def tokenizer_function(example):
  return tokenizer(example, padding="max_length", truncation=True)



In [14]:
dataset.astype({'sentence_A': 'string', 'sentence_B': 'string', 'label_new': 'float64' }).dtypes

sentence_A     string
sentence_B     string
label_new     float64
dtype: object

In [15]:
import numpy as np

In [16]:
test = dataset['label_new']
type(test)

pandas.core.series.Series

In [31]:
df_A = dataset['sentence_A'].apply(tokenizer_function)
df_B = dataset['sentence_B'].apply(tokenizer_function)
df_label = dataset['label_new']
df_tokenized =  pd.concat([df_A, df_B, df_label], axis=1)

In [32]:
df_tokenized.head()

,sentence_A,sentence_B,label_new
0,"[input_ids, attention_mask]","[input_ids, attention_mask]",0.989013
1,"[input_ids, attention_mask]","[input_ids, attention_mask]",0.960834
2,"[input_ids, attention_mask]","[input_ids, attention_mask]",0.990987
3,"[input_ids, attention_mask]","[input_ids, attention_mask]",0.967705
4,"[input_ids, attention_mask]","[input_ids, attention_mask]",0.975873


In [83]:
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset

tfds_token = datasets.Dataset(df_tokenized)

NameError: ignored

In [81]:
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset

df = df_tokenized
dataset = ds.dataset(pa.Table.from_pandas(df).to_batches())



ArrowInvalid: ignored

In [ ]:
### convert to Huggingface dataset
hg_dataset = Dataset(pa.Table.from_pandas(df))

In [67]:
df_tokenized.to_csv(index=False)

In [69]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [72]:
from google.colab import drive

drive.mount('/content/drive')
path = '/content/drive/My Drive/df_sick_tokenized.csv'

with open(path, 'w', encoding = 'utf-8-sig') as f:
  df_tokenized.to_csv(f)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [74]:
from datasets import Dataset
import pandas as pd

In [78]:
dataset = Dataset.from_pandas(df_tokenized)

ArrowInvalid: ignored

In [63]:
type(df_tokenized['sentence_A'])

pandas.core.series.Series

In [35]:
train_labels = df_tokenized['label_new'].to_numpy().flatten()
train_labels

array([0.98901306, 0.96083428, 0.9909867 , ..., 0.73105858, 0.73105858,
       0.73105858])

In [49]:
features = df_tokenized.columns
features = list(features)
features

['sentence_A', 'sentence_B', 'label_new']

In [58]:
#features = np.asarray(features).astype(np.int)
features = np.asarray(features).astype(np.string_)
#X_train=np.asarray(X_train).astype(np.int)
#y_train=np.asarray(y_train).astype(np.int)

In [62]:
#TF dataset
dataset = tf.data.Dataset.from_tensor_slices((df_tokenized, features))


ValueError: ignored

In [ ]:
for features_tensor, target_tensor in training_dataset:
    print(f'features:{features_tensor} target:{target_tensor}')

In [38]:
'''
features = ['feature1', 'feature2', 'feature3']
print(training_df)

training_dataset = (
    tf.data.Dataset.from_tensor_slices(
        (
            tf.cast(training_df[features].values, tf.float32),
            tf.cast(training_df['target'].values, tf.int32)
        )
    )
)

for features_tensor, target_tensor in training_dataset:
    print(f'features:{features_tensor} target:{target_tensor}')
  '''


"\nfeatures = ['feature1', 'feature2', 'feature3']\nprint(training_df)\n\ntraining_dataset = (\n    tf.data.Dataset.from_tensor_slices(\n        (\n            tf.cast(training_df[features].values, tf.float32),\n            tf.cast(training_df['target'].values, tf.int32)\n        )\n    )\n)\n\nfor features_tensor, target_tensor in training_dataset:\n    print(f'features:{features_tensor} target:{target_tensor}')\n  "

In [19]:
import pandas as pd
import tensorflow as tf


Features,labels = df_tokenized,df_tokenized.pop('label_new')

dataset = tf.data.Dataset.from_tensor_slices((dict(Features), labels))
iterator = dataset.make_initializable_iterator()
next_element = iterator.get_next()

with tf.Session() as sess:
  sess.run(iterator.initializer)
  print(sess.run(next_element))
  print(sess.run(next_element))

ValueError: ignored

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2)

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000)) 
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000)) 
full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["test"]

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer")

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model, args=training_args, train_dataset=small_train_dataset, eval_dataset=small_eval_dataset
)

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 1000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 375


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=375, training_loss=0.28083123779296876, metrics={'train_runtime': 551.4558, 'train_samples_per_second': 5.44, 'train_steps_per_second': 0.68, 'total_flos': 789333166080000.0, 'train_loss': 0.28083123779296876, 'epoch': 3.0})

In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]